In [1]:
import json
from collections import defaultdict


In [3]:
INPUT_JSON = "../../data/interim/poi_all_interim.json"
OUTPUT_JSON = "poi_with_id_region.json"


In [4]:
def map_category(poi_type: str) -> str:
    if poi_type in {"attraction", "leisure", "shopping"}:
        return "tourspot"
    elif poi_type == "cafe":
        return "cafe"
    elif poi_type == "restaurant":
        return "restaurant"
    else:
        return "etc"


In [5]:
with open(INPUT_JSON, "r", encoding="utf-8") as f:
    pois = json.load(f)

print(f"Loaded POIs: {len(pois)}")


Loaded POIs: 3350


In [6]:
category_counters = defaultdict(int)


In [7]:
for poi in pois:
    poi_type = poi.get("type")

    # category 결정
    category = map_category(poi_type)
    poi["category"] = category

    # category별 id 증가 (파일 순서 기준)
    category_counters[category] += 1
    poi["id"] = category_counters[category]

    # region 고정
    poi["region"] = "seoul"


In [8]:
for poi in pois[:5]:
    print({
        "poi_id": poi.get("poi_id"),
        "type": poi.get("type"),
        "category": poi.get("category"),
        "id": poi.get("id"),
        "region": poi.get("region")
    })


{'poi_id': '2456536', 'type': 'attraction', 'category': 'tourspot', 'id': 1, 'region': 'seoul'}
{'poi_id': '1603149', 'type': 'attraction', 'category': 'tourspot', 'id': 2, 'region': 'seoul'}
{'poi_id': '1602451', 'type': 'attraction', 'category': 'tourspot', 'id': 3, 'region': 'seoul'}
{'poi_id': '3076460', 'type': 'attraction', 'category': 'tourspot', 'id': 4, 'region': 'seoul'}
{'poi_id': '2733970', 'type': 'attraction', 'category': 'tourspot', 'id': 5, 'region': 'seoul'}


In [10]:
TOP_KEYS = ["id", "category", "region"]


In [11]:
def reorder_poi_fields(poi: dict) -> dict:
    ordered = {}

    # 1) 맨 위로 올릴 키들
    for key in TOP_KEYS:
        if key in poi:
            ordered[key] = poi[key]

    # 2) 나머지 키들 (기존 순서 유지)
    for key, value in poi.items():
        if key not in ordered:
            ordered[key] = value

    return ordered


In [12]:
pois_reordered = [reorder_poi_fields(poi) for poi in pois]


In [14]:
with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(pois_reordered, f, ensure_ascii=False, indent=2)

print("Saved with reordered fields")


Saved with reordered fields


In [16]:
pois[0]

{'poi_id': '2456536',
 'type': 'attraction',
 'sub_type': '관광단지',
 'name': '강남 마이스 관광특구',
 'address': '서울특별시 강남구 영동대로 513',
 'gu_name': '강남구',
 'image': 'http://tong.visitkorea.or.kr/cms/resource/13/3464913_image2_1.jpg',
 'location': {'lat': 37.5118092746, 'lng': 127.0591318945},
 'description': '일찍이 무역센터가 자리 잡으며 국제 교류의 메카로 손꼽혀온 강남 삼성동 일대는, 세계에서 모여든 최첨단 비즈니스맨에게 한국을 알리는 창구 역할을 해왔다. 여기에 1990년대부터 강남 특유의 젊은 문화가 유입되면서 무역센터 일대는 비즈니스의 장이자 젊은 문화의 랜드마크라는 복합적인 성격을 띠며 발전했다. \n\n2014년 싸이의 ‘강남스타일’ 열풍과 더불어 국제교류 비즈니스 산업과 한류문화의 공생을 강화하기 위해 이 일대가 강남 마이스 관광특구로 지정되었다. 강남 마이스 관광특구는 전시와 컨벤션, 세미나, 국제 전시회, 콘퍼런스 등이 이어지는 코엑스를 중심으로 자리 잡고 있고, 백화점, 호텔, 면세점, 도심공항터미널이 유기적으로 모여 있어, 국내뿐만이 아니라 외국인 관광객이 끊이지 않는다. 강남 마이스 관광특구 일대는 한국판 타임스퀘어로 조성되어 최첨단 디스플레이를 활용해 홀로그램과 증강현실을 체험하고, 건물 외벽에 설치된 초대형 LED 전광판에서 표출되는 화려한 미디어아트와 광고를 감상할 수 있다.',
 'attributes': {'duration': '2~3시간',
  'activity': {'label': '적당히 걷는', 'level': 3},
  'photospot': True,
  'indoor_outdoor': '복합',
  'keywords': ['전시회', '컨벤션', '미디어아트', '최첨단 기술 체험', '도심 야경'

place id(int)*, type→category(tourspot, restaurant, cafe)*, region*, name*, address*, duration*, description, images, latitude*, longitude* 필요


In [ ]:
def extract_place_fields(poi: dict) -> dict:
    return {
        # "poi_id": poi["id"],                                # poi_id
        "place_id": poi["id"],                              # int *
        "category": poi["category"],                        # tourspot / cafe / restaurant *
        "region": poi["region"],                            # *
        "name": poi.get("name"),                            # *
        "address": poi.get("address"),                      # *
        "duration": poi.get("attributes", {}).get("duration"),  # *
        "description": poi.get("description"),              # optional
        "images": (
            [poi["image"]] if poi.get("image") else []
        ),                                                   # optional (list)
        "latitude": poi.get("location", {}).get("lat"),     # *
        "longitude": poi.get("location", {}).get("lng"),    # *
    }


In [18]:
cleaned_pois = [extract_place_fields(poi) for poi in pois]


In [19]:
with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(cleaned_pois, f, ensure_ascii=False, indent=2)

print("Saved with reordered fields")


Saved with reordered fields
